### Schedule Optimization with CPLEX and Python
<img src="ScheduleOptimsation_RedLobsterMoonfruit.jpg" alt="ScheduleOptimsation" width="500"/>

In [ ]:
!pip3 install jupyter_contrib_nbextensions
!jupyter contrib nbextension install

In [ ]:
# !pip install RISE #one time-set up for creating Jupyter slideshows

## Step 1: Describe the problem

The first step in modeling the problem is to write a natural language description of the problem, identifying the decision variables and the constraints on these variables.

Writing a natural language description of this problem requires to answer these questions:

- What is the known information in this problem ?
- What are the decision variables or unknowns in this problem ?
- What are the constraints on these variables ?
- What is the objective ?

_What is the known information in this problem ?_

There are ten house building tasks, each with a given duration. For each task, there is a list of tasks that must be completed before the task can start. Some tasks also have costs associated with an early start date or late end date.

_What are the decision variables or unknowns in this problem ?_

The unknowns are the date that each task will start. The cost is determined by the assigned start dates.

_What are the constraints on these variables ?_

In this case, each constraint specifies that a particular task may not begin until one or more given tasks have been completed.

_What is the objective ?_

The objective is to minimize the cost incurred through earliness and tardiness costs.

## Step 2: Model Setup

When using a cloud solving solution, the following attributes must be set with appropriate values:

In [ ]:
url = None
key = None

### Declaration of engine
A scheduling model starts with the declaration of the engine as follows:

In [ ]:
# !pip install docplex #one time-set up

In [ ]:
import sys
from docplex.cp.model import *

In [ ]:
#check environment

from docplex.mp.environment import Environment
env = Environment()
env.print_information()

In [ ]:
# create the model container:
mdl1 = CpoModel()

## Step 3: Declare the interval variables
In the model, each task is represented by an interval variables. Each variable represents the unknown information, the scheduled interval for each activity. After the model is executed, the values assigned to these interval variables will represent the solution to the problem.

The declaration of necessary interval variables is done as follows:

| Task | Duration | Preceding tasks |
| --- | --- | --- |
| masonry | 35 |  |
| capentry | 15 | masonry |
| COMPLETE THIS 

In [ ]:
masonry = mdl1.interval_var(size=35)
carpentry = mdl1.interval_var(size=15)
plumbing = mdl1.interval_var(size=40)
ceiling = mdl1.interval_var(size=15)
roofing = mdl1.interval_var(size=5)
painting = mdl1.interval_var(size=10)
windows = mdl1.interval_var(size=5)
facade = mdl1.interval_var(size=10)
garden = mdl1.interval_var(size=5)
moving = mdl1.interval_var(size=5)

## Step 4: Add the precedence constraints

In this example, certain tasks can start only after other tasks have been completed. CP Optimizer allows to express constraints involving temporal relationships between pairs of interval variables using precedence constraints.

Precedence constraints are used to specify when an interval variable must start or end with respect to the start or end time of another interval variable.

The following types of precedence constraints are available; if a and b denote interval variables, both interval variables are present, and delay is a number or integer expression (0 by default), then:

end_before_end(a, b, delay) constrains at least the given delay to elapse between the end of a and the end of b. It imposes the inequality endTime(a) + delay <= endTime(b).
end_before_start(a, b, delay) constrains at least the given delay to elapse between the end of a and the start of b. It imposes the inequality endTime(a) + delay <= startTime(b).
end_at_end(a, b, delay) constrains the given delay to separate the end of a and the end of ab. It imposes the equality endTime(a) + delay == endTime(b).
end_at_start(a, b, delay) constrains the given delay to separate the end of a and the start of b. It imposes the equality endTime(a) + delay == startTime(b).
start_before_end(a, b, delay) constrains at least the given delay to elapse between the start of a and the end of b. It imposes the inequality startTime(a) + delay <= endTime(b).
start_before_start(a, b, delay) constrains at least the given delay to elapse between the start of act1 and the start of act2. It imposes the inequality startTime(a) + delay <= startTime(b).
start_at_end(a, b, delay) constrains the given delay to separate the start of a and the end of b. It imposes the equality startTime(a) + delay == endTime(b).
start_at_start(a, b, delay) constrains the given delay to separate the start of a and the start of b. It imposes the equality startTime(a) + delay == startTime(b).
If either interval a or b is not present in the solution, the constraint is automatically satisfied, and it is as if the constraint was never imposed.

For our model, precedence constraints can be added with the following code:

In [ ]:
mdl1.add( mdl1.end_before_start(masonry, carpentry) )
mdl1.add( mdl1.end_before_start(masonry, plumbing) )
mdl1.add( mdl1.end_before_start(masonry, ceiling) )
mdl1.add( mdl1.end_before_start(carpentry, roofing) )
mdl1.add( mdl1.end_before_start(ceiling, painting) )
mdl1.add( mdl1.end_before_start(roofing, windows) )
mdl1.add( mdl1.end_before_start(roofing, facade) )
mdl1.add( mdl1.end_before_start(plumbing, facade) )
mdl1.add( mdl1.end_before_start(roofing, garden) )
mdl1.add( mdl1.end_before_start(plumbing, garden) )
mdl1.add( mdl1.end_before_start(windows, moving) )
mdl1.add( mdl1.end_before_start(facade, moving) )
mdl1.add( mdl1.end_before_start(garden, moving) )
mdl1.add( mdl1.end_before_start(painting, moving) )

## Step 5: Add the objective

The objective function to be minimized can be written as follows:

In [ ]:
obj = mdl1.minimize(  400 * mdl1.max([mdl1.end_of(moving) - 100, 0]) 
                    + 200 * mdl1.max([25 - mdl1.start_of(masonry), 0]) 
                    + 300 * mdl1.max([75 - mdl1.start_of(carpentry), 0]) 
                    + 100 * mdl1.max([75 - mdl1.start_of(ceiling), 0]) )
mdl1.add(obj)

## Step 6: Solve the model and display the solution

In [ ]:
# Solve the model
print("\nSolving model....")
msol1 = mdl1.solve(url=url, key=key, TimeLimit=20)
print("done")

In [ ]:
print("Cost will be " + str(msol1.get_objective_values()[0]))

In [ ]:
var_sol = msol1.get_var_solution(masonry)
print("Masonry : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(carpentry)
print("Carpentry : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(plumbing)
print("Plumbing : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(ceiling)
print("Ceiling : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(roofing)
print("Roofing : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(painting)
print("Painting : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(windows)
print("Windows : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(facade)
print("Facade : {}..{}".format(var_sol.get_start(), var_sol.get_end()))
var_sol = msol1.get_var_solution(moving)
print("Moving : {}..{}".format(var_sol.get_start(), var_sol.get_end()))

Graphical display of the same result is available with:

In [ ]:
#setting up graphical display
import docplex.cp.utils_visu as visu
import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 15, 3

In [ ]:
wt = msol1.get_var_solution(masonry)   
visu.interval(wt, 'lightblue', 'masonry')   
wt = msol1.get_var_solution(carpentry)   
visu.interval(wt, 'lightblue', 'carpentry')
wt = msol1.get_var_solution(plumbing)   
visu.interval(wt, 'lightblue', 'plumbing')
wt = msol1.get_var_solution(ceiling)   
visu.interval(wt, 'lightblue', 'ceiling')
wt = msol1.get_var_solution(roofing)   
visu.interval(wt, 'lightblue', 'roofing')
wt = msol1.get_var_solution(painting)   
visu.interval(wt, 'lightblue', 'painting')
wt = msol1.get_var_solution(windows)   
visu.interval(wt, 'lightblue', 'windows')
wt = msol1.get_var_solution(facade)   
visu.interval(wt, 'lightblue', 'facade')
wt = msol1.get_var_solution(moving)   
visu.interval(wt, 'lightblue', 'moving')
visu.show()

The overall cost is 5000 and moving will be completed by day 110.

from https://ibmdecisionoptimization.github.io/tutorials/html/Scheduling_Tutorial.html